In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
%cd /content/drive/MyDrive/works/client-project/stock-inventory-business-case/project-group.zip (Unzipped Files)

/content/drive/MyDrive/rubixe-works/client-project/stock-inventory-business-case/project-group.zip (Unzipped Files)


In [ ]:
%pwd

'/content/drive/MyDrive/works/client-project/stock-inventory-business-case/project-group.zip (Unzipped Files)'

# Step 1: Business Case - Spareparts inventory forecasting
The business case is based on inventory management. Keeping
Inventory of spare in various service centre to the market demand is
always a challenge as most service centres spends significant amount
in spare parts inventory costs. In spite of this, availability of spare
parts is been one of the problem areas.

### Project Goal
Create Predictive model for inventory forecasting so that
service centre achieve JIT standards.

*JIT:* The just-in-time (JIT) inventory system is a management strategy that aligns raw-material orders from suppliers directly with production schedules.

### Project goal approches
Here, project goal is to forecast inventory spare-parts. So it is might be more suitabl to forecast spare-parts name and its count on a given date. But this dataset seems to be complicated because of having 502 output classes, also their count is not given. Therefore we will go through two modelling approaches mentioned below and select high performing modelling approach. 
- **Approach 1:** Build a multi-class classification model using classification algorithms for top 7 classes.
- **Approach 2:** Build a 3 time-series model for top 3 classes to forecast spare-parts counts on given date.


# Step 2: Data Collection
Data is taken from 'project_service_data' database provided by client. It is an SQL data base and we using SQL commands to access the data. 

In [ ]:
#! pip install pymysql

In [ ]:
from sqlalchemy import create_engine
import numpy
import pandas as pd

In [ ]:
## Creating connection to data base

db_host = '18.136.157.135'  # IP address of the server
username = 'dm_usdata_sql'
user_passwd = '37z<49REb&mKnl4AV!vJ'
db_name = 'project_service_data'

conn = create_engine('mysql+pymysql://'+username+':'+user_passwd+'@'+db_host+'/'+db_name)
conn.table_names()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  if __name__ == '__main__':


['service_data']

In [ ]:
query = 'SELECT * FROM service_data'
data = pd.read_sql(query, conn)
data

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL
...,...,...,...,...,...,...,...
28477,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER
28478,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE
28479,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER
28480,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL


In [ ]:
## saving dataset into local storage after fetching from data base.

save_data = data.copy()
save_data.to_csv('service_data.csv', index=False)

In [ ]:
data = pd.read_csv('service_data.csv')
data.head()

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL


**NOTE:** Uploaded documents doesn’t contain separate dataset files for approach 1 and 2 because of exceeding maximum number of documents (5 documents). Only have 'service_data.csv'. So you can generate those separate dataset by running codes in this notebook file. 
# Step 3 : Identifying the independent and dependent variable.
#### Input Features
- **invoice_date:** invoice date represents the time-stamped time and date on which the goods have been billed and the transaction officially recorded. Therefore, the invoice date has essential information regarding payment, as it dictates the credit duration and due date of the bill.
- **job_card_date:** The job card is meant to ensure that the technician working on the vehicle, has a detailed 'patient' file which will help ensure that every job is completed professionally and every vehicle receives the technician's full attention and application. Job card is the communication language between vehicle inspector and worker.
- **business_partner_name:** Individual or small shope dealer for whome the work need to be done.
- **vehicle_no:** Indicates vehicle registration. Have registered vehicles of almost all States in India.
- **vehicle_model:** Vehicle brand-model name.
- **current_km_reading:** number of kilometer already run by the vehicle.

#### Output variable
- **invoice_line_text:** Spare-parts mentioned in invoice; name of spare-part.

# Step 3 : EDA
#### Basic checks

In [ ]:
data

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL
...,...,...,...,...,...,...,...
28477,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER
28478,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE
28479,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER
28480,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28482 entries, 0 to 28481
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   invoice_date           28482 non-null  object
 1   job_card_date          28482 non-null  object
 2   business_partner_name  28482 non-null  object
 3   vehicle_no             28482 non-null  object
 4   vehicle_model          28482 non-null  object
 5   current_km_reading     28482 non-null  int64 
 6   invoice_line_text      28448 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [ ]:
data.isnull().sum()

invoice_date              0
job_card_date             0
business_partner_name     0
vehicle_no                0
vehicle_model             0
current_km_reading        0
invoice_line_text        34
dtype: int64

output variable 'invoice_line_text' has some missing values

In [ ]:
data.describe()

,current_km_reading
count,28482.000000
mean,19348.003827
std,25246.722306
min,0.000000
25%,3988.000000
50%,12420.500000
75%,27905.000000
max,610112.000000


In [ ]:
data.nunique()

invoice_date              555
job_card_date             553
business_partner_name    1010
vehicle_no                846
vehicle_model              28
current_km_reading       3474
invoice_line_text         502
dtype: int64

In [ ]:
pd.set_option('display.max_rows', None)
data.vehicle_no.value_counts()

In [ ]:
data.vehicle_model.value_counts()

BAJAJ PULSAR 150            8633
BAJAJ AVENGER STREET 220    4212
BAJAJ PULSAR 220            2655
BAJAJ PULSAR 180            2252
BAJAJ PULSAR NS 200         2204
BAJAJ CT 100                1432
BAJAJ DISCOVER 125          1346
BAJAJ AVENGER STREET        1112
BAJAJ PLATINA                844
BAJAJ V150                   686
BAJAJ DISCOVER               679
BAJAJ PULSAR LS135           520
BAJAJ PULSAR AS 200          421
BAJAJ PULSAR NS 160          364
BAJAJ PULSAR RS 200          322
BAJAJ DISCOVER 150           284
BAJAJ AVENGER STREET 180     122
BAJAJ DISCOVER 135           115
BAJAJ V125                    95
BAJAJ V                       54
BAJAJ CALIBER                 40
BAJAJ XCD 125                 27
BAJAJ V15                     23
BAJAJ XCD 135                 13
WAVE                          10
BAJAJ DOMINAR 400 ABS          7
BAJAJ WIND 125                 6
BAJAJ BOXER CT                 4
Name: vehicle_model, dtype: int64

In [ ]:
data.invoice_date.value_counts()

01-12-18    179
04-12-18    167
30-11-18    148
05-11-18    136
05-09-18    133
           ... 
27-12-17      5
18-11-18      5
18-02-18      4
09-07-17      1
30-05-17      1
Name: invoice_date, Length: 555, dtype: int64

In [ ]:
data.job_card_date.value_counts()

01-12-18    179
04-12-18    167
30-11-18    148
05-11-18    136
05-09-18    133
           ... 
15-01-18      5
18-11-18      5
18-02-18      4
09-07-17      1
30-05-17      1
Name: job_card_date, Length: 553, dtype: int64

In [ ]:
data.business_partner_name.value_counts()

In [ ]:
len(data.invoice_line_text.value_counts())

502

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data.invoice_line_text.value_counts()

ENGINE OIL              3802
CHAIN LUBRICATION       3441
GENERAL SERVICE         2142
AIR FILTER              1715
3M OIL                  1628
                        ... 
REGISTER EXPENSES          1
MAGNET COVER PACKING       1
CHAIN PAD                  1
DISK PLATE                 1
HELMET                     1
Name: invoice_line_text, Length: 502, dtype: int64

In [ ]:
## removing rows having missing output values
data.loc[data.invoice_line_text.isnull()]

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
545,16-06-17,13-06-17,diliXXXXXXXXXX,KA53S3XXXX,BAJAJ V,2615,NaN
632,15-06-17,14-06-17,naveXXXXXXXXXX,KA53W3XXXX,BAJAJ DISCOVER 150,25677,NaN
635,15-06-17,14-06-17,naveXXXXXXXXXX,KA53W3XXXX,BAJAJ DISCOVER 150,25677,NaN
661,15-06-17,14-06-17,kailXXXXXXXXXX,KA01HNXXXX,BAJAJ PULSAR 150,8679,NaN
769,16-06-17,16-06-17,hariXXXXXXXXXX,KA53ESXXXX,BAJAJ AVENGER STREET,457,NaN
2139,21-07-17,21-07-17,diliXXXXXXXXXX,KA53S3XXXX,BAJAJ V,12457,NaN
2579,01-08-17,01-08-17,hariXXXXXXXXXX,KA53EQXXXX,BAJAJ PULSAR 150,9677,NaN
2613,03-08-17,03-08-17,diliXXXXXXXXXX,KA53S3XXXX,BAJAJ V,251447,NaN
10809,27-02-18,27-02-18,PRAVXXXXXXXXXX,KA04JBXXXX,BAJAJ PULSAR 150,24447,NaN
10810,27-02-18,27-02-18,PRAVXXXXXXXXXX,KA04JBXXXX,BAJAJ PULSAR 150,24447,NaN


In [ ]:
data = data.dropna()
data

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL
...,...,...,...,...,...,...,...
28477,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER
28478,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE
28479,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER
28480,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL


In [ ]:
data.shape

(28448, 7)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
#! pip install sweetviz

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
import sweetviz as sv

#### Univariate analysis

In [ ]:
data_analysis = sv.analyze(data)
data_analysis.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


As we mentioned above, since dataset consists of features having more than 100 unique values, it is difficult to visualize on graphs.

So we will go through some data preperation and feature selection process on this existing dataset, then try to visualize on graph. In feature selection process we only select few number of top output classes instead on 502 classes because it is difficult to build model with all 502 classes. 

## Data preperation
Preparing data for both approaches 1 and 2. 

**Set a date features from "job_card_date":**
Prepare a date feature column for time-series modelling.

In [ ]:
data

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL
...,...,...,...,...,...,...,...
28443,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER
28444,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE
28445,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER
28446,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL


In [ ]:
## checking dates arrangement in 'job_card_date' feature: before sorting
## format: D-M-Y
## start: 30-05-17
## end: 06-01-19

In [ ]:
data.job_card_date.nunique()

553

In [ ]:
data.sort_values('job_card_date', inplace=True)

In [ ]:
data

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
9580,01-02-18,01-02-18,sanjXXXXXXXXXX,KA03JDXXXX,BAJAJ PULSAR 180,19993,CLUTCH COVER GASKET
9551,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,CLUTCH OVERHUAL
9550,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,HEAD CAP O RING
9549,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,CLUTCH ASSEMBLY
9548,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,AIR FILTER
...,...,...,...,...,...,...,...
28027,31-12-18,31-12-18,hendXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,5885,AIR FILTER
28028,31-12-18,31-12-18,hendXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,5885,SPROCKET RUBBER
28029,31-12-18,31-12-18,hendXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,5885,GENERAL SERVICE
28069,31-12-18,31-12-18,muraXXXXXXXXXX,KA03JQXXXX,BAJAJ PULSAR NS 200,12136,CHAIN LUBRICATION


In [ ]:
## checking dates arrangement in 'job_card_date' feature: after sorting
## start: 01-02-18
## end: 31-12-18
## it is not proper

In [ ]:
data1 = data.copy()

In [ ]:
## since "job_card_date" is not in proper date format, converting it into Pandas datetime format.

data1['job_card_date_dt'] = pd.to_datetime(data1['job_card_date'])
data1

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,job_card_date_dt
9580,01-02-18,01-02-18,sanjXXXXXXXXXX,KA03JDXXXX,BAJAJ PULSAR 180,19993,CLUTCH COVER GASKET,2018-01-02
9551,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,CLUTCH OVERHUAL,2018-01-02
9550,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,HEAD CAP O RING,2018-01-02
9549,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,CLUTCH ASSEMBLY,2018-01-02
9548,01-02-18,01-02-18,kasiXXXXXXXXXX,KA53X1XXXX,BAJAJ PULSAR LS135,50589,AIR FILTER,2018-01-02
...,...,...,...,...,...,...,...,...
28027,31-12-18,31-12-18,hendXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,5885,AIR FILTER,2018-12-31
28028,31-12-18,31-12-18,hendXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,5885,SPROCKET RUBBER,2018-12-31
28029,31-12-18,31-12-18,hendXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,5885,GENERAL SERVICE,2018-12-31
28069,31-12-18,31-12-18,muraXXXXXXXXXX,KA03JQXXXX,BAJAJ PULSAR NS 200,12136,CHAIN LUBRICATION,2018-12-31


In [ ]:
data1.job_card_date_dt.min()

Timestamp('2017-01-06 00:00:00')

In [ ]:
data1.job_card_date_dt.max()

Timestamp('2019-06-01 00:00:00')

In [ ]:
data1.sort_values('job_card_date_dt', inplace=True)
data1

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,job_card_date_dt
57,01-06-17,01-06-17,SATHXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,523,CHAIN LUBRICATION,2017-01-06
62,01-06-17,01-06-17,vinoXXXXXXXXXX,KA51EVXXXX,BAJAJ AVENGER STREET 220,2750,CHAIN LUBRICATION,2017-01-06
88,01-06-17,01-06-17,kesaXXXXXXXXXX,KA03HSXXXX,BAJAJ PULSAR 150,39421,DISC LEVER,2017-01-06
61,01-06-17,01-06-17,vinoXXXXXXXXXX,KA51EVXXXX,BAJAJ AVENGER STREET 220,2750,FORK BEND,2017-01-06
82,01-06-17,01-06-17,kesaXXXXXXXXXX,KA03HSXXXX,BAJAJ PULSAR 150,39421,SIDE SHIELD,2017-01-06
...,...,...,...,...,...,...,...,...
28420,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,ENGINE OIL,2019-06-01
28446,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL,2019-06-01
28424,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,FORK OIL SEAL,2019-06-01
28432,06-01-19,06-01-19,PRITXXXXXXXXXX,KA53EWXXXX,BAJAJ PULSAR NS 160,2914,POLISH,2019-06-01


In [ ]:
## after converting into datetime format
## format: Y-M-D
## start: 2017-01-06
## end: 2019-06-01
## Date column arrangement is proper

In [ ]:
data1.reset_index(inplace=True)

In [ ]:
data1.drop(['index'], axis=1, inplace=True)

In [ ]:
data1

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,job_card_date_dt
0,01-06-17,01-06-17,SATHXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,523,CHAIN LUBRICATION,2017-01-06
1,01-06-17,01-06-17,vinoXXXXXXXXXX,KA51EVXXXX,BAJAJ AVENGER STREET 220,2750,CHAIN LUBRICATION,2017-01-06
2,01-06-17,01-06-17,kesaXXXXXXXXXX,KA03HSXXXX,BAJAJ PULSAR 150,39421,DISC LEVER,2017-01-06
3,01-06-17,01-06-17,vinoXXXXXXXXXX,KA51EVXXXX,BAJAJ AVENGER STREET 220,2750,FORK BEND,2017-01-06
4,01-06-17,01-06-17,kesaXXXXXXXXXX,KA03HSXXXX,BAJAJ PULSAR 150,39421,SIDE SHIELD,2017-01-06
...,...,...,...,...,...,...,...,...
28443,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,ENGINE OIL,2019-06-01
28444,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL,2019-06-01
28445,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,FORK OIL SEAL,2019-06-01
28446,06-01-19,06-01-19,PRITXXXXXXXXXX,KA53EWXXXX,BAJAJ PULSAR NS 160,2914,POLISH,2019-06-01


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28448 entries, 9580 to 28021
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   invoice_date           28448 non-null  object
 1   job_card_date          28448 non-null  object
 2   business_partner_name  28448 non-null  object
 3   vehicle_no             28448 non-null  object
 4   vehicle_model          28448 non-null  object
 5   current_km_reading     28448 non-null  int64 
 6   invoice_line_text      28448 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.7+ MB


In [ ]:
data1.invoice_line_text.value_counts()[:10]

ENGINE OIL           3802
CHAIN LUBRICATION    3441
GENERAL SERVICE      2142
AIR FILTER           1715
3M OIL               1628
CONSUMABLES          1595
POLISH               1245
DISC OIL              991
BRAKE SHOE            965
OIL FILTER            821
Name: invoice_line_text, dtype: int64

## Selecting data
Here, feature "invoice_line_text" tells which spare parts being ordered and used. This feature has 502 unique class names. So it is difficult to consider all those classes. Hence we are taking top seven classes (mostly repeated classes) for approach 1 and top three classes for aproach 2.

Top classes and its counts

ENGINE OIL 3802,
CHAIN LUBRICATION 3441,
GENERAL SERVICE 2142,
AIR FILTER 1715, 
3M OIL 1628, 
CONSUMABLES 1595,
POLISH 1245.

### Taking top 7 classes for Approach 1 (classification modelling)

In [ ]:
invoice_line_text_selected = ['ENGINE OIL', 'CHAIN LUBRICATION', 'GENERAL SERVICE', 'AIR FILTER', '3M OIL', 
                              'CONSUMABLES', 'POLISH']

In [ ]:
data1_selected = data1.loc[data1.invoice_line_text.isin(invoice_line_text_selected)==True]

In [ ]:
data1_selected

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,job_card_date_dt
0,01-06-17,01-06-17,SATHXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,523,CHAIN LUBRICATION,2017-01-06
1,01-06-17,01-06-17,vinoXXXXXXXXXX,KA51EVXXXX,BAJAJ AVENGER STREET 220,2750,CHAIN LUBRICATION,2017-01-06
8,01-06-17,01-06-17,diliXXXXXXXXXX,ka 53 XXXX,BAJAJ V,45421,GENERAL SERVICE,2017-01-06
10,01-06-17,01-06-17,SATHXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,523,3M OIL,2017-01-06
11,01-06-17,01-06-17,PRASXXXXXXXXXX,KA53ERXXXX,BAJAJ PULSAR 150,5500,3M OIL,2017-01-06
...,...,...,...,...,...,...,...,...
28441,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,GENERAL SERVICE,2019-06-01
28442,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,CHAIN LUBRICATION,2019-06-01
28443,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,ENGINE OIL,2019-06-01
28446,06-01-19,06-01-19,PRITXXXXXXXXXX,KA53EWXXXX,BAJAJ PULSAR NS 160,2914,POLISH,2019-06-01


In [ ]:
data1_selected = data1_selected.reset_index(drop=True)
data1_selected

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text,job_card_date_dt
0,01-06-17,01-06-17,SATHXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,523,CHAIN LUBRICATION,2017-01-06
1,01-06-17,01-06-17,vinoXXXXXXXXXX,KA51EVXXXX,BAJAJ AVENGER STREET 220,2750,CHAIN LUBRICATION,2017-01-06
2,01-06-17,01-06-17,diliXXXXXXXXXX,ka 53 XXXX,BAJAJ V,45421,GENERAL SERVICE,2017-01-06
3,01-06-17,01-06-17,SATHXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR 150,523,3M OIL,2017-01-06
4,01-06-17,01-06-17,PRASXXXXXXXXXX,KA53ERXXXX,BAJAJ PULSAR 150,5500,3M OIL,2017-01-06
...,...,...,...,...,...,...,...,...
15563,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,GENERAL SERVICE,2019-06-01
15564,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,CHAIN LUBRICATION,2019-06-01
15565,06-01-19,06-01-19,dharXXXXXXXXXX,KA41L7XXXX,BAJAJ PULSAR 150,91448,ENGINE OIL,2019-06-01
15566,06-01-19,06-01-19,PRITXXXXXXXXXX,KA53EWXXXX,BAJAJ PULSAR NS 160,2914,POLISH,2019-06-01


In [ ]:
data1_selected.invoice_line_text.value_counts()

ENGINE OIL           3802
CHAIN LUBRICATION    3441
GENERAL SERVICE      2142
AIR FILTER           1715
3M OIL               1628
CONSUMABLES          1595
POLISH               1245
Name: invoice_line_text, dtype: int64

In [ ]:
data_top7_class = data1_selected.copy()
data_top7_class.to_csv('data_top7_class.csv', index=False)

## this data set will be used for Approach 1

### Taking top 3 classes for Approach 2 (time-series modelling)

Here we create three dataset for each 3 output class. Each data set have two columns;
- Date: Same as 'job_card_date_dt' feature.
- Quantity/count: This is new feature, specifying count of spare-parts on a given date. It is calculated as total number of spare-parts for a given unique 'Date'.

In [ ]:
unique_date = data1_selected.job_card_date_dt.unique()
len(unique_date)

552

In [ ]:
## creating a nested dictionary that have outer keys as date and inner dictionary with keys as spare-parts name and values as its count
## on that day.  

invoice_line_text_selected = ['ENGINE OIL', 'CHAIN LUBRICATION', 'GENERAL SERVICE']
data1_item_count_in_date = {}
for i in range(len(unique_date)):
  feat = {}
  for j in invoice_line_text_selected:
    count = 0
    count = len(data1_selected.loc[(data1_selected.invoice_line_text==j) & (data1_selected.job_card_date_dt==unique_date[i])])
    feat[j]=count
  data1_item_count_in_date[unique_date[i]]=feat

In [ ]:
dict(list(data1_item_count_in_date.items())[:5])

{numpy.datetime64('2017-01-06T00:00:00.000000000'): {'ENGINE OIL': 8,
  'CHAIN LUBRICATION': 5,
  'GENERAL SERVICE': 3},
 numpy.datetime64('2017-01-07T00:00:00.000000000'): {'ENGINE OIL': 12,
  'CHAIN LUBRICATION': 9,
  'GENERAL SERVICE': 9},
 numpy.datetime64('2017-01-08T00:00:00.000000000'): {'ENGINE OIL': 6,
  'CHAIN LUBRICATION': 7,
  'GENERAL SERVICE': 4},
 numpy.datetime64('2017-01-09T00:00:00.000000000'): {'ENGINE OIL': 7,
  'CHAIN LUBRICATION': 5,
  'GENERAL SERVICE': 3},
 numpy.datetime64('2017-01-10T00:00:00.000000000'): {'ENGINE OIL': 3,
  'CHAIN LUBRICATION': 2,
  'GENERAL SERVICE': 2}}

In [ ]:
len(data1_item_count_in_date)

552

In [ ]:
data1_item_count_in_date[unique_date[-1]]

{'ENGINE OIL': 4, 'CHAIN LUBRICATION': 4, 'GENERAL SERVICE': 2}

Preparing separate time-series dataset for all three classes

In [ ]:
def get_count_list(class_name):
  count_list = []
  for i in unique_date:
    count = data1_item_count_in_date[i][class_name]
    count_list.append(count)
  return count_list

**"ENGINE OIL" time series data**

In [ ]:
engineoil_df = pd.DataFrame({'Date': unique_date, 'Engine Oil Quantity': get_count_list('ENGINE OIL')})
engineoil_df

,Date,Engine Oil Quantity
0,2017-01-06,8
1,2017-01-07,12
2,2017-01-08,6
3,2017-01-09,7
4,2017-01-10,3
...,...,...
547,2019-02-01,6
548,2019-03-01,6
549,2019-04-01,11
550,2019-05-01,11


In this dataframe, last five records seems to be improper in date format compared all others. So let swap month as day and day as month.

In [ ]:
engineoil_df.Date.replace(to_replace = [numpy.datetime64('2019-02-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-03-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-04-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-05-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-06-01T00:00:00.000000000')],
                 value = [numpy.datetime64('2019-01-02T00:00:00.000000000'), 
                         numpy.datetime64('2019-01-03T00:00:00.000000000'),
                         numpy.datetime64('2019-01-04T00:00:00.000000000'),
                         numpy.datetime64('2019-01-05T00:00:00.000000000'),
                         numpy.datetime64('2019-01-06T00:00:00.000000000')], inplace=True)

In [ ]:
engineoil_df

,Date,Engine Oil Quantity
0,2017-01-06,8
1,2017-01-07,12
2,2017-01-08,6
3,2017-01-09,7
4,2017-01-10,3
...,...,...
547,2019-01-02,6
548,2019-01-03,6
549,2019-01-04,11
550,2019-01-05,11


In [ ]:
engineoil_df.to_csv('engine_oil_tsdata.csv',index=False)

**"CHAIN LUBRICATION" time series data**

In [ ]:
chainlubrication_df = pd.DataFrame({'Date': unique_date, 
                                    'Chain Lubrication Quantity': get_count_list('CHAIN LUBRICATION')})

In [ ]:
chainlubrication_df.Date.replace(to_replace = [numpy.datetime64('2019-02-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-03-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-04-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-05-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-06-01T00:00:00.000000000')],
                 value = [numpy.datetime64('2019-01-02T00:00:00.000000000'), 
                         numpy.datetime64('2019-01-03T00:00:00.000000000'),
                         numpy.datetime64('2019-01-04T00:00:00.000000000'),
                         numpy.datetime64('2019-01-05T00:00:00.000000000'),
                         numpy.datetime64('2019-01-06T00:00:00.000000000')], inplace=True)

In [ ]:
chainlubrication_df

,Date,Chain Lubrication Quantity
0,2017-01-06,5
1,2017-01-07,9
2,2017-01-08,7
3,2017-01-09,5
4,2017-01-10,2
...,...,...
547,2019-01-02,5
548,2019-01-03,4
549,2019-01-04,10
550,2019-01-05,9


In [ ]:
chainlubrication_df.to_csv('chain_lubrication_tsdata.csv',index=False)

**"GENERAL SERVICE" time series data**

In [ ]:
generalservice_df = pd.DataFrame({'Date': unique_date, 'General Service Quantity': get_count_list('GENERAL SERVICE')})

In [ ]:
generalservice_df.Date.replace(to_replace = [numpy.datetime64('2019-02-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-03-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-04-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-05-01T00:00:00.000000000'),
                                       numpy.datetime64('2019-06-01T00:00:00.000000000')],
                 value = [numpy.datetime64('2019-01-02T00:00:00.000000000'), 
                         numpy.datetime64('2019-01-03T00:00:00.000000000'),
                         numpy.datetime64('2019-01-04T00:00:00.000000000'),
                         numpy.datetime64('2019-01-05T00:00:00.000000000'),
                         numpy.datetime64('2019-01-06T00:00:00.000000000')], inplace=True)

In [ ]:
generalservice_df

,Date,General Service Quantity
0,2017-01-06,3
1,2017-01-07,9
2,2017-01-08,4
3,2017-01-09,3
4,2017-01-10,2
...,...,...
547,2019-01-02,7
548,2019-01-03,6
549,2019-01-04,6
550,2019-01-05,6


In [ ]:
generalservice_df.to_csv('general_service_tsdata.csv',index=False)